In [9]:
import time
from pathlib import Path
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StringType, LongType, IntegerType, DoubleType, BooleanType, StructType, StructField, TimestampType
import os
import logging
import warnings
import datetime

import urllib
import zipfile

# Suppress Python warnings
warnings.filterwarnings("ignore")

# Suppress Java/Spark logging
logging.getLogger("py4j").setLevel(logging.ERROR)
logging.getLogger("pyspark").setLevel(logging.ERROR)
logging.getLogger("org.apache.spark").setLevel(logging.ERROR)
logging.getLogger("org.apache.hadoop").setLevel(logging.ERROR)

In [10]:
# Verify Java
os.system("java -version")

0

In [11]:
TARGET_DATE = "2025-09-09"
data_dir = Path("./data_cleaned")
posts_path = str(data_dir / "*_posts.csv")
comments_path = str(data_dir / "*_comments.csv")
spark = SparkSession.builder.getOrCreate()
print(spark.version)

3.5.0


In [12]:
print(f"\n--- Loading and filtering Reddit data for {TARGET_DATE} ---")
posts_path = str(data_dir / "*_posts.csv")
posts_df_string = spark.read.csv(posts_path, header=True, inferSchema=False)
posts_transformed_df = posts_df_string.withColumn(
    "timestamp", F.to_timestamp(F.from_unixtime(F.expr("try_cast(created_utc AS LONG)")))
).withColumn(
    "score", F.expr("try_cast(score AS INT)")
).withColumn(
    "num_comments", F.expr("try_cast(num_comments AS INT)")
)
reddit_with_date = posts_transformed_df.withColumn("date", F.to_date(F.col("timestamp")))
reddit_single_day_df = reddit_with_date.filter(F.col("date") == F.to_date(F.lit(TARGET_DATE)))
# reddit_single_day_df.cache()

reddit_count = reddit_single_day_df.count()
if reddit_count == 0:
    print(f"Warning: No Reddit posts found for {TARGET_DATE}. The analysis will be empty.")
else:
    print(f"Found {reddit_count:,} Reddit posts for {TARGET_DATE}.")


--- Loading and filtering Reddit data for 2025-09-09 ---
Found 1,255 Reddit posts for 2025-09-09.


In [13]:
print(f"\n--- Batch Acquiring GDELT data (Sept 01 - Sept 25) ---")

gdelt_dir = Path("./gdelt_data")
os.makedirs(gdelt_dir, exist_ok=True)

# Define date range
start_date = datetime.date(2025, 9, 1)
end_date = datetime.date(2025, 9, 25)
delta = datetime.timedelta(days=1)

current_date = start_date
files_downloaded = 0

while current_date <= end_date:
    date_str = current_date.strftime("%Y%m%d")
    gdelt_zip_filename = f"{date_str}.export.CSV.zip"
    gdelt_csv_filename = f"{date_str}.export.CSV"
    gdelt_zip_path = gdelt_dir / gdelt_zip_filename
    gdelt_csv_path = gdelt_dir / gdelt_csv_filename
    gdelt_url = f"http://data.gdeltproject.org/events/{gdelt_zip_filename}"

    # Check if CSV already exists
    if not gdelt_csv_path.exists():
        print(f"[{date_str}] Downloading...", end=" ", flush=True)
        try:
            # Download
            urllib.request.urlretrieve(gdelt_url, gdelt_zip_path)
            
            # Unzip
            with zipfile.ZipFile(gdelt_zip_path, 'r') as zip_ref:
                zip_ref.extractall(gdelt_dir)
            
            # Cleanup Zip
            os.remove(gdelt_zip_path)
            print("✅ Done.")
            files_downloaded += 1
        except Exception as e:
            print(f"❌ Failed: {e}")
    else:
        # print(f"[{date_str}] Already exists.") # Uncomment to see skipped files
        pass

    current_date += delta

print(f"\nBatch download complete. {files_downloaded} new files acquired.")

# --- LOAD ALL FILES INTO SPARK ---
# Spark can read a directory or a list of files as a single DataFrame!
print("Loading GDELT data into Spark...")

# Get list of all CSVs in the directory
all_gdelt_files = [str(p) for p in gdelt_dir.glob("*.export.CSV")]

if all_gdelt_files:
    # Read without schema first (Robust Method)
    # Spark automatically unions all these files into one big DataFrame
    gdelt_raw_df = spark.read.csv(all_gdelt_files, sep='\t', inferSchema=False)
    
    # Select columns by index (Standard GDELT Layout)
    # SQLDATE=1, Actor1Name=5, EventCode=26, GoldsteinScale=30, AvgTone=34
    gdelt_df = gdelt_raw_df.select(
        F.col("_c1").alias("SQLDATE"),
        F.col("_c5").alias("Actor1Name"),
        F.col("_c26").alias("EventCode"),
        F.col("_c30").cast("double").alias("GoldsteinScale"),
        F.col("_c34").cast("double").alias("AvgTone")
    ).withColumn("date", F.to_date(F.col("SQLDATE"), "yyyyMMdd"))
    
    # Cache this larger dataset
    gdelt_df.cache()
    total_events = gdelt_df.count()
    print(f"✅ Successfully loaded {len(all_gdelt_files)} days of GDELT data.")
    print(f"📊 Total Global Events in DataFrame: {total_events:,}")
else:
    print("❌ No GDELT files found.")


--- Batch Acquiring GDELT data (Sept 01 - Sept 25) ---

Batch download complete. 0 new files acquired.
Loading GDELT data into Spark...
✅ Successfully loaded 25 days of GDELT data.
📊 Total Global Events in DataFrame: 2,993,530


In [14]:
if gdelt_df and reddit_single_day_df.count() > 0:
    print(f"\n--- Deriving Comparative Insights for {TARGET_DATE} ---")
    
    # Insight 1: (No changes needed, the result is already insightful)
    print("\nInsight 1: Overall Sentiment Comparison (News vs. Reddit)")
    gdelt_avg_sentiment = gdelt_df.agg(F.avg("AvgTone").alias("avg_news_sentiment")).first()['avg_news_sentiment']
    reddit_avg_score = reddit_single_day_df.agg(F.avg("score").alias("avg_reddit_score")).first()['avg_reddit_score']
    print(f"  - Average GDELT News Sentiment (AvgTone): {gdelt_avg_sentiment:.2f}")
    print(f"  - Average Reddit Post Score: {reddit_avg_score:.2f}")

    # Insight 2: Find Mentions of Top GDELT Actors in Reddit Titles (with case-insensitivity)
    print(f"\nInsight 2: Mentions of Top GDELT Actors in Reddit Titles")
    generic_actors = ["PROTESTER", "GOVERNMENT", "POLICE", "REBEL", "MILITARY", "CITIZEN", "MEDIA", "OPPOSITION", "GOV", "JUD"]
    top_actors = gdelt_df.filter(
        F.col("Actor1Name").isNotNull() & (~F.col("Actor1Name").isin(generic_actors))
    ).groupBy("Actor1Name").count().orderBy(F.col("count").desc()).limit(5)
    top_actors_list = [row.Actor1Name for row in top_actors.collect()]
    
    if not top_actors_list:
        print("No significant non-generic actors found in the GDELT sample.")
    else:
        print(f"Top 5 Filtered GDELT Actors in sample: {top_actors_list}")
        mention_counts = []
        for actor in top_actors_list:
            # THE FIX: Use F.lower() for a case-insensitive search
            count = reddit_single_day_df.filter(F.locate(actor.lower(), F.lower(F.col("title"))) > 0).count()
            mention_counts.append((actor, count))
        print("Mentions of these actors in Reddit titles:")
        for actor, count in mention_counts:
            print(f"  - '{actor}': {count} mentions")

    # Insight 3: Compare Thematic Focus (with robust stop word removal)
    print(f"\nInsight 3: Thematic Focus Comparison (GDELT Events vs. Reddit Keywords)")
    print("  - Top 10 GDELT Event Types:")
    gdelt_df.groupBy("EventCode").count().orderBy(F.col("count").desc()).show(10, truncate=False)
    
    # THE FIX: Use a standard list of stop words for a more reliable filter
    print("  - Top 10 Reddit Title Keywords (after robust stop word removal):")
    # Get the default stop words from the ML library
    from pyspark.ml.feature import StopWordsRemover
    stopwords = StopWordsRemover.loadDefaultStopWords("english")
    
    reddit_keywords = reddit_single_day_df.withColumn("word", F.explode(F.split(F.lower(F.col("title")), "[^a-zA-Z]"))) \
        .filter(~F.col("word").isin(stopwords)) \
        .filter(F.col("word") != "") \
        .groupBy("word").count().orderBy(F.col("count").desc())
    reddit_keywords.show(10, truncate=False)


--- Deriving Comparative Insights for 2025-09-09 ---

Insight 1: Overall Sentiment Comparison (News vs. Reddit)
  - Average GDELT News Sentiment (AvgTone): -1.99
  - Average Reddit Post Score: 451.43

Insight 2: Mentions of Top GDELT Actors in Reddit Titles
Top 5 Filtered GDELT Actors in sample: ['USA', 'GBR', 'COP', 'BUS', 'EDU']
Mentions of these actors in Reddit titles:
  - 'USA': 14 mentions
  - 'GBR': 0 mentions
  - 'COP': 5 mentions
  - 'BUS': 7 mentions
  - 'EDU': 6 mentions

Insight 3: Thematic Focus Comparison (GDELT Events vs. Reddit Keywords)
  - Top 10 GDELT Event Types:
+---------+------+
|EventCode|count |
+---------+------+
|010      |240046|
|042      |233748|
|043      |218977|
|051      |195411|
|040      |191740|
|020      |173823|
|190      |115235|
|036      |114339|
|046      |95323 |
|173      |94551 |
+---------+------+
only showing top 10 rows

  - Top 10 Reddit Title Keywords (after robust stop word removal):
+--------+-----+
|word    |count|
+--------+-----+

In [15]:
spark.stop()
print("\n✅ Spark session stopped. Script finished.")


✅ Spark session stopped. Script finished.
